# Developing a Hybrid Recommender System

# Business problem
### Let's make predictions for the user whose ID is given, using the item-based and user-based recomennder methods.
### Let's take 5 suggestions from the user-based model, and 5 suggestions from the item-based model, and finally, let's make 10 suggestions from 2 models.

# Variables
### There are many different tables in this dataset, but there are 2 CSV files to use.

#### movie.csv
* movieId - Unique movie number
* title - movie name
#### rating.csv
* userid - Unique user number
* movieId - Unique movie number
* rating - the rating given to the movie by the user
* timestamp - review date

# Importing the libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

# Importing and combining dataframes

In [2]:
movie = pd.read_csv('/kaggle/input/movie-rating-hybrid/movie.csv')
rating = pd.read_csv('/kaggle/input/movie-rating-hybrid/rating.csv')
df = movie.merge(rating, how='left', on='movieId')
df.columns = [col.lower() for col in df.columns]
df

,movieid,title,genres,userid,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,2015-03-30 19:32:59
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,2015-03-30 19:48:08
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,2015-03-30 19:56:32
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,2015-03-30 19:57:46


# Data preparation

### Let's calculate the total number of people who voted for each movie and remove the movies with less than 1000 votes from the data set.

In [3]:
comment_counts = pd.DataFrame(df['title'].value_counts())
rare_movies = comment_counts[comment_counts['title'] <= 1000].index
common_movies = df[~df['title'].isin(rare_movies)]
common_movies

,movieid,title,genres,userid,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
19985698,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,28195.0,4.0,2014-09-22 20:52:18
19985699,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,51334.0,3.0,2014-09-23 15:53:39
19985700,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,120575.0,2.5,2014-10-08 14:23:39
19985701,114240,Aladdin (1992),Adventure|Animation|Children|Comedy|Fantasy,124998.0,2.5,2014-09-20 22:16:14


### So, the number of observations reduced from 20000797 to 17766015 rows.

### Let's create a pivot table for the dataframe with the userids in the index, the movie names in the columns and the ratings as values.

In [4]:
user_movie_df = common_movies.pivot_table(index=['userid'], columns=['title'], values='rating')
user_movie_df

title     'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  \
userid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
1.0                      NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN   
2.0                      NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN   
3.0                      NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 5.0   
4.0                      NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN   
5.0                      NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN   
...                      ...                          ...                             ...                            ...                                ...               ...                    ...                                                ...                         ...                  ...                      ...               ...                    ...                       ...          ...                ...               ...              ...                                 ...   
138489.0                 NaN            

### Let's functionalize all the above operations

In [5]:
"""
def user_movie_datafreme(th=1000):
    import pandas as pd
    movie = pd.read_csv('/kaggle/input/movie-rating-hybrid/movie.csv')
    rating = pd.read_csv('/kaggle/input/movie-rating-hybrid/rating.csv')
    dataframe = movie.merge(rating, how='left', on='movieId')
    dataframe.columns = [col.lower() for col in dataframe.columns]
    comment_counts = pd.DataFrame(dataframe['title'].value_counts())
    rare_movies = comment_counts[comment_counts['title'] <= th].index
    common_movies = dataframe[~dataframe['title'].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=['userid'], columns=['title'], values='rating')
    return user_movie_df

user_movie_df = user_movie_datafreme()
user_movie_df.head()
"""

"\ndef user_movie_datafreme(th=1000):\n    import pandas as pd\n    movie = pd.read_csv('/kaggle/input/movie-rating-hybrid/movie.csv')\n    rating = pd.read_csv('/kaggle/input/movie-rating-hybrid/rating.csv')\n    dataframe = movie.merge(rating, how='left', on='movieId')\n    dataframe.columns = [col.lower() for col in dataframe.columns]\n    comment_counts = pd.DataFrame(dataframe['title'].value_counts())\n    rare_movies = comment_counts[comment_counts['title'] <= th].index\n    common_movies = dataframe[~dataframe['title'].isin(rare_movies)]\n    user_movie_df = common_movies.pivot_table(index=['userid'], columns=['title'], values='rating')\n    return user_movie_df\n\nuser_movie_df = user_movie_datafreme()\nuser_movie_df.head()\n"

### Let's Determine the Movies Watched by the User to Suggest

### Let's choose a random user id

In [6]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=19).values)
random_user

123192

### Let's create a new dataframe named random_user_df consisting of observation units of the selected user.

In [7]:
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df

title     'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  \
userid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
123192.0                 NaN                          NaN                             NaN                            NaN                                NaN               NaN                    NaN                                                NaN                         NaN                  NaN                      NaN               NaN                    NaN                       NaN          NaN                NaN               NaN              NaN                                 NaN   

title     2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  20,000 Leagues Under the Sea (1954)  200 Cigarettes (1999)  2001: A Space Odyssey (1968)  2010: The Year We Make Contact (1984)  2012 (2009)  2046 (2004)  21 (2008)  21 Grams (2003)  21 Jump Street (2012)  24 Hour Party People (2002)  25th Hour (2002)  27 Dresses (2008)  28 Days (2000)  28 Days Later (2002)  28 Weeks Later (2007)  3 Ninjas (1992)  3-Iron (Bin-jip) (2004)  30 Days of Night (2007)  \
userid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
123192.0                          NaN                                                NaN                                      NaN                    NaN                           NaN                                    NaN          NaN          NaN        NaN              NaN                    NaN                          NaN               NaN                NaN             NaN                   NaN                    NaN              NaN                      NaN                      NaN   

title     300 (2007)  3000 Miles to Graceland (2001)  39 Steps, The (1935)  3:10 to Yuma (2007)  40 Days and 40 Nights (2002)  40-Year-Old Virgin, The (2005)  400 Blows, The (Les quatre cents coups) (1959)  42 Up (1998)  48 Hrs. (1982)  50 First Dates (2004)  50/50 (2011)  54 (1998)  6th Day, The (2000)  7th Voyage of Sinbad, The (1958)  8 1/2 (8½) (1963)  8 Heads in a Duffel Bag (1997)  8 Mile (2002)  8MM (1999)  9 (2009)  9 1/2 Weeks (Nine 1/2 Weeks) (1986)  A-Team, The (2010)  \
userid                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
123192.0         NaN                             Na

In [8]:
random_user_df.shape

(1, 3159)

### Let's assign the movies that the selected user has voted to a list called movies_watched

In [9]:
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['Ace Ventura: Pet Detective (1994)',
 'Ace Ventura: When Nature Calls (1995)',
 'Addams Family Values (1993)',
 'Aladdin (1992)',
 'Apollo 13 (1995)',
 'Babe (1995)',
 'Batman (1989)',
 'Batman Forever (1995)',
 'Beauty and the Beast (1991)',
 'Beverly Hills Cop III (1994)',
 'Bridges of Madison County, The (1995)',
 'Casper (1995)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Clear and Present Danger (1994)',
 'Client, The (1994)',
 'Cliffhanger (1993)',
 'Coneheads (1993)',
 'Congo (1995)',
 'Crimson Tide (1995)',
 'Dances with Wolves (1990)',
 'Disclosure (1994)',
 'Dumb & Dumber (Dumb and Dumber) (1994)',
 'French Kiss (1995)',
 'I.Q. (1994)',
 'Jumanji (1995)',
 'Junior (1994)',
 'Legends of the Fall (1994)',
 'Lion King, The (1994)',
 'Man of the House (1995)',
 'Miracle on 34th Street (1994)',
 'Natural Born Killers (1994)',
 'Net, The (1995)',
 'Nine Months (1995)',
 'Operation Dumbo Drop (1995)',
 'Outbreak (1995)',
 'Pulp Fiction (1994)',
 'Quiz Show (1994)',
 

In [10]:
len(movies_watched)

50

### Let's Access Data and Ids of Other Users Watching the Same Movies

### Let's select the columns of the movies watched by the selected user from user_movie_df and create a new dataframe named movies_watched_df

In [11]:
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Aladdin (1992),Apollo 13 (1995),Babe (1995),Batman (1989),Batman Forever (1995),Beauty and the Beast (1991),Beverly Hills Cop III (1994),"Bridges of Madison County, The (1995)",Casper (1995),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),"Client, The (1994)",Cliffhanger (1993),Coneheads (1993),Congo (1995),Crimson Tide (1995),Dances with Wolves (1990),Disclosure (1994),Dumb & Dumber (Dumb and Dumber) (1994),French Kiss (1995),I.Q. (1994),Jumanji (1995),Junior (1994),Legends of the Fall (1994),"Lion King, The (1994)",Man of the House (1995),Miracle on 34th Street (1994),Natural Born Killers (1994),"Net, The (1995)",Nine Months (1995),Operation Dumbo Drop (1995),Outbreak (1995),Pulp Fiction (1994),Quiz Show (1994),"River Wild, The (1994)","Santa Clause, The (1994)",Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)",Snow White and the Seven Dwarfs (1937),Species (1995),Tom and Huck (1995),Tommy Boy (1995),Toy Story (1995),True Lies (1994),Waterworld (1995),When a Man Loves a Woman (1994),While You Were Sleeping (1995)
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,3.5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN
4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN


### Let's create a new dataframe named user_movie_count that contains the information of how many movies each user has watched and create a new df

In [12]:
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count.head()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ['userId', 'movie_count']
user_movie_count

,userId,movie_count
0,1.0,4
1,2.0,1
2,3.0,4
3,4.0,5
4,5.0,11
...,...,...
138488,138489.0,2
138489,138490.0,6
138490,138491.0,1
138491,138492.0,0


### Let's see those who watched 60 percent or more of the movies that the selected user voted for as similar users and create a list called users_same_movies from the ids of these users.

In [13]:
users_same_movies = user_movie_count[user_movie_count['movie_count'] >= (len(movies_watched) * 0.6)]['userId']
users_same_movies

12            13.0
25            26.0
28            29.0
33            34.0
90            91.0
            ...   
138414    138415.0
138430    138431.0
138473    138474.0
138482    138483.0
138483    138484.0
Name: userId, Length: 6188, dtype: float64

# Determining the Users to be Suggested and Most Similar Users

### Let's fi[[](http://)](http://)lter the movies_watched_df dataframe to find the ids of the users that are similar to the selected user in the user_same_movies list.

In [14]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)], random_user_df[movies_watched]])
final_df.head()

title,Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Addams Family Values (1993),Aladdin (1992),Apollo 13 (1995),Babe (1995),Batman (1989),Batman Forever (1995),Beauty and the Beast (1991),Beverly Hills Cop III (1994),"Bridges of Madison County, The (1995)",Casper (1995),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),"Client, The (1994)",Cliffhanger (1993),Coneheads (1993),Congo (1995),Crimson Tide (1995),Dances with Wolves (1990),Disclosure (1994),Dumb & Dumber (Dumb and Dumber) (1994),French Kiss (1995),I.Q. (1994),Jumanji (1995),Junior (1994),Legends of the Fall (1994),"Lion King, The (1994)",Man of the House (1995),Miracle on 34th Street (1994),Natural Born Killers (1994),"Net, The (1995)",Nine Months (1995),Operation Dumbo Drop (1995),Outbreak (1995),Pulp Fiction (1994),Quiz Show (1994),"River Wild, The (1994)","Santa Clause, The (1994)",Seven (a.k.a. Se7en) (1995),"Shawshank Redemption, The (1994)",Snow White and the Seven Dwarfs (1937),Species (1995),Tom and Huck (1995),Tommy Boy (1995),Toy Story (1995),True Lies (1994),Waterworld (1995),When a Man Loves a Woman (1994),While You Were Sleeping (1995)
userid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
13.0,3.0,3.0,3.0,1.0,5.0,1.0,3.0,3.0,2.0,4.0,NaN,NaN,3.0,3.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,3.0,NaN,3.0,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN,NaN,5.0,5.0,2.0,NaN,3.0,5.0,5.0,NaN,NaN,NaN,NaN,4.0,2.0,NaN,NaN,3.0
26.0,3.0,3.0,3.0,3.0,4.0,NaN,3.0,3.0,5.0,3.0,NaN,NaN,NaN,3.0,NaN,3.0,3.0,2.0,3.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,4.0,3.0,NaN,NaN,3.0,5.0,4.0,NaN,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,3.0
29.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,3.0,4.0,3.0,NaN,3.0,4.0,3.0,NaN,4.0,3.0,3.0,4.0,4.0,3.0,3.0,NaN,NaN,3.0,3.0,4.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,5.0,5.0,2.0,NaN,4.0,4.0,5.0,4.0,3.0,NaN,4.0,NaN,3.0,3.0,NaN,3.0
34.0,3.0,NaN,4.0,4.0,5.0,3.0,4.0,3.0,5.0,NaN,NaN,3.0,3.0,NaN,NaN,3.0,3.0,NaN,5.0,2.0,NaN,3.0,4.0,4.0,3.0,NaN,NaN,4.0,NaN,3.0,NaN,3.0,3.0,3.0,NaN,3.0,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,NaN,4.0
91.0,2.5,2.0,2.5,NaN,3.5,NaN,4.0,3.0,NaN,2.5,NaN,NaN,3.0,3.0,2.5,3.0,2.5,1.0,3.5,4.5,NaN,2.0,NaN,2.5,3.5,1.5,3.5,4.5,NaN,NaN,NaN,2.5,2.5,2.5,4.0,3.5,3.5,2.5,3.5,4.0,4.0,3.5,1.5,NaN,2.5,4.0,3.0,1.5,NaN,2.5


### Let's create a new corr_df dataframe where users' correlations with each other will be found.

In [15]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df = pd.DataFrame(corr_df, columns=['corr'])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df

,user_id_1,user_id_2,corr
0,81473.0,108515.0,-0.903717
1,35166.0,85760.0,-0.878222
2,71994.0,7961.0,-0.873231
3,33761.0,130803.0,-0.852204
4,25353.0,7159.0,-0.850616
...,...,...,...
15845810,36310.0,7058.0,0.976508
15845811,108792.0,158.0,0.978958
15845812,137686.0,39619.0,0.979050
15845813,123192.0,123192.0,1.000000


### Let's create a new dataframe named top_users by filtering out users that have high correlation (over 0.65) with the selected user

In [16]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df['corr'] >= 0.65)][['user_id_2', 'corr']].reset_index(drop=True)
top_users.sort_values(by='corr', ascending=False)
top_users.columns = ['userId', 'corr']
top_users

,userId,corr
0,138431.0,0.659505
1,56330.0,0.660948
2,125385.0,0.663271
3,125674.0,0.670914
4,125674.0,0.670914
5,80544.0,0.672282
6,107587.0,0.678412
7,32568.0,0.688309
8,24161.0,0.696673
9,72771.0,0.700628


### Let's merge the top_users dataframe with the rating dataset

In [17]:
top_users_ratings = top_users.merge(rating[['userId', 'movieId', 'rating']], how='inner')
top_users_ratings

,userId,corr,movieId,rating
0,138431.0,0.659505,1,5.0
1,138431.0,0.659505,2,5.0
2,138431.0,0.659505,10,3.0
3,138431.0,0.659505,11,4.0
4,138431.0,0.659505,21,3.0
...,...,...,...,...
1491,123192.0,1.000000,590,5.0
1492,123192.0,1.000000,592,3.0
1493,123192.0,1.000000,594,4.0
1494,123192.0,1.000000,595,5.0


# Calculating Weighted Average Recommendation Score and Keeping Top 5 Movies 

### Let's create a new variable named weighted_rating, which is the product of the corr and rating values of each user.

In [18]:
top_users_ratings['weighted_rating'] = top_users_ratings['rating'] * top_users_ratings['corr']

### Let's create a new dataframe called recommendation_df that contains the movie id and the average value of the weighted_ratings of all users for each movie

In [19]:
recommendation_df = top_users_ratings.groupby('movieId').agg({'weighted_rating': 'mean'})
recommendation_df = recommendation_df.reset_index()
recommendation_df

,movieId,weighted_rating
0,1,3.509461
1,2,2.952232
2,3,2.017463
3,5,1.807769
4,6,2.035236
...,...,...
492,2822,2.016846
493,3114,3.361410
494,3147,2.689128
495,3156,2.689128


### Select movies with a weighted rating greater than 3 in recommendation_df and sort by weighted rating and save the top 5 observations as movies_to_be_recommend.

In [20]:
movies_to_be_recommend = recommendation_df[recommendation_df['weighted_rating'] > 3].sort_values('weighted_rating', ascending=False)
movies_to_be_recommend

,movieId,weighted_rating
264,554,4.000000
180,381,3.892588
168,364,3.630197
0,1,3.509461
245,527,3.499423
62,150,3.498908
145,318,3.421618
51,104,3.392060
418,1674,3.361410
423,1693,3.361410


In [21]:
movies_to_be_recommend = movies_to_be_recommend.merge(movie[['movieId', 'title']])
movies_to_be_recommend['title'].head()

0               Trial by Jury (1994)
1    When a Man Loves a Woman (1994)
2              Lion King, The (1994)
3                   Toy Story (1995)
4            Schindler's List (1993)
Name: title, dtype: object

# Item-Based Recommendation

### Let's make an item-based suggestion based on the name of the movie that the user last watched and gave the highest rating

In [22]:
user = 123192

### Let's read movie, rating datasets again

In [23]:
movie = pd.read_csv('/kaggle/input/movie-rating-hybrid/movie.csv')
rating = pd.read_csv('/kaggle/input/movie-rating-hybrid/rating.csv')
df = movie.merge(rating, how='left', on='movieId')
df.columns = [col.lower() for col in df.columns]
df

,movieid,title,genres,userid,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000792,131254,Kein Bund für's Leben (2007),Comedy,79570.0,4.0,2015-03-30 19:32:59
20000793,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570.0,4.0,2015-03-30 19:48:08
20000794,131258,The Pirates (2014),Adventure,28906.0,2.5,2015-03-30 19:56:32
20000795,131260,Rentun Ruusu (2001),(no genres listed),65409.0,3.0,2015-03-30 19:57:46


### Let's get the id of the movie with the most recent score from the movies that the selected user gave 5 points

In [24]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
selected_movie = df[(df["userid"] == user) & (df['rating'] == 5)]
most_current_movie = selected_movie[selected_movie['timestamp'] == selected_movie['timestamp'].max()]
most_current_movie

,movieid,title,genres,userid,rating,timestamp
2423423,350,"Client, The (1994)",Drama|Mystery|Thriller,123192.0,5.0,1996-05-15 08:57:58


### Let's filter the user_movie_df dataframe created in the User based recommendation section according to the selected movie id.

In [25]:
filtered_movie = user_movie_df['Client, The (1994)']      # according to the title

### Let's find the correlation of the selected movie with the other movies using the filtered dataframe and rank

In [26]:
user_movie_df.corrwith(filtered_movie).sort_values(ascending=False).head(6)

title
Client, The (1994)                1.000000
Pelican Brief, The (1993)         0.572074
Chamber, The (1996)               0.560818
Firm, The (1993)                  0.540865
Blind Side, The  (2009)           0.532993
Sleeping with the Enemy (1991)    0.514645
dtype: float64

### Let's recommend the top 5 movies excluding the selected movie itself

In [27]:
first_five_movies = pd.DataFrame(user_movie_df.corrwith(filtered_movie).sort_values(ascending=False))
first_five_movies = first_five_movies.reset_index()
first_five_movies.columns = ['title', 'corr']
first_five_movies[first_five_movies['title'] != 'Client, The (1994)']['title'].head()

1         Pelican Brief, The (1993)
2               Chamber, The (1996)
3                  Firm, The (1993)
4           Blind Side, The  (2009)
5    Sleeping with the Enemy (1991)
Name: title, dtype: object

### Consequently, we recommended 5 movies to the selected random user based on both user-based and item-based collaborative filtering methods.

# Thank you for checking my notebook!